In [ ]:
%load_ext jupyter_black

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import pulp as pl
import cplex

from copy import deepcopy

# Graph

1. Density is the proportion of edges in a full connected graph. A dense graph has $n(n-1)$ edges where $n$ is the number of nodes.
2. `maximum_flow` from networkx has a linear trend w.r.t. the graph density.
3. Column generation has an exponential trend w.r.t. density (keep a density < 0.3 !!!)

In [ ]:
min_weight = 5
max_weight = 15
n_nodes = 5 * 10**3
n_edges = 249_950
density = n_edges / (n_nodes * (n_nodes - 1))
print("density = ", density)
source, destination = 0, 1  # don't need to randomize (graph is randomized)

## Creates a random valuated graph

In [ ]:
def modify_weights_graph(graph, ctr_lists, new_weights):
    """Utility function for setting weights (dual problem of shortest path)"""
    nx.set_edge_attributes(graph, 0, name="weight")

    nx.set_edge_attributes(
        graph,
        values={edge: dual for edge, dual in zip(ctr_lists, new_weights)},
        name="weight",
    )

In [ ]:
graph = nx.gnm_random_graph(n_nodes, n_edges, directed=True)
dual_graph = deepcopy(graph)

for (u, v) in graph.edges():
    graph.edges[u, v]["weight"] = np.random.uniform(min_weight, max_weight)

In [ ]:
adj_matrix = nx.adjacency_matrix(graph)
adj_matrix.shape

N.B.: The problem of maximum flow is equivalent to maximum path generation **if we consider all possible paths**

# Column Generation

In [ ]:
def get_n_paths(graph, source, destination, max_iter=16):
    """Get the max_iter paths of all_simple_paths generator: Graph: source --|> destination"""
    list_ = []
    for (index, path) in enumerate(
        nx.all_simple_paths(graph, source, destination, cutoff=10)
    ):
        list_.append(path)
        if index >= max_iter - 1:
            return list_
    return list_

### Affichage de $n$ chemins

In [ ]:
len(get_n_paths(graph, source, destination, max_iter=50))

## Master

### Initialise the master problem

In [ ]:
def add_paths(graph, lpProb, paths, path_count, path2idx):
    """Add paths in lpProb (new variables Xp and new constraints or terms in constraints Ce)"""

    # For every path add one variable, and its constraints
    for path in paths:

        # add a variable
        prov = pl.LpVariable("X" + str(path_count), 0, max_weight, pl.LpContinuous)
        lpProb.objective.addterm(prov, 1)

        # for each edge on the path, search the associated constraint (indexed by edge)
        for index in range(len(path) - 1):
            index_ctr = str((path[index], path[index + 1]))
            if index_ctr in lpProb.constraints:
                lpProb.constraints[index_ctr].addterm(prov, 1)  # add term in constraint
            else:
                lpProb.constraints[index_ctr] = (
                    prov <= graph.edges[(path[index], path[index + 1])]
                )
                ctr_lists.append((path[index], path[index + 1]))

        path2idx[path_count] = str(path)
        path_count += 1
    return (path_count, path2idx)

### Dual problem

In [ ]:
def get_duals(lpPb):
    """Get the dual variables from a problem Pulp"""
    return [c.pi for _, c in list(lpPb.constraints.items())]

### Iterative procedure

In [ ]:
lpProb = pl.LpProblem(
    name="path_generation",
    sense=pl.LpMaximize,
)
lpProb += 0  # initiate the objective function to 0 (necessary line of code)

path2idx = (
    {}
)  # useful to know the final paths at the end and calculate the shortest path for dual problem
ctr_lists = []
path_count = 0

path_count, path2idx = add_paths(
    graph,
    lpProb,
    get_n_paths(graph, source, destination, max_iter=1),
    path_count,
    path2idx,
)
solver = pl.PULP_CBC_CMD(msg=False)
result = lpProb.solve(solver)

In [ ]:
go_on = True
reduced_cost_arr = np.array([])
reduced_cost = 1

while go_on:
    # search of path q (dual problem)
    duals = get_duals(lpProb)
    modify_weights_graph(dual_graph, ctr_lists, duals)
    path_q = nx.shortest_path(dual_graph, source, destination, weight="weight")

    # compute reduced cost for dual

    for e in range(len(path_q) - 1):
        try:
            reduced_cost -= lpProb.constraints[str((path_q[e], path_q[e + 1]))].pi
        except KeyError:  # <|-- constraint does not exist (so dual == 0)

            pass

    # stop procedure ?
    if reduced_cost > 0:  # add path q in lp Problem
        reduced_cost_arr = np.append(reduced_cost_arr, reduced_cost)
        path_count, path2idx = add_paths(graph, lpProb, [path_q], path_count, path2idx)
        lpProb.solve(solver)
    else:
        go_on = False

In [ ]:
reduced_cost_arr

In [ ]:
flow = 0
for var in lpProb.variables():
    flow += var.value()
print(flow)

In [ ]:
paths = {
    path2idx[int(str(var)[1:])]: var.value()
    for var in lpProb.variables()
    if var.value() > 0
}
paths